![](run.jpg)

# Setup

In [ ]:
from arcgis import GIS

In [ ]:
gis = GIS('https://python.playground.esri.com/portal', 'arcgis_python')

In [ ]:
counties_item = gis.content.search('USA Counties', 'Feature Layer', sort_field='avgRating',
                                   outside_org=True)[0]

In [ ]:
counties_item

In [ ]:
counties = counties_item.layers[0]

In [ ]:
ca_counties = counties.query(where="STATE_NAME='California'")

In [ ]:
counties_df = ca_counties.df

In [ ]:
sd_county = counties_df[counties_df.NAME == 'San Diego']

In [ ]:
# query the shape of San Diego county and set it's spatial reference
sd_geom = sd_county.SHAPE.iloc[0]
sd_geom.spatial_reference = counties.container.properties.spatialReference

In [ ]:
# Digital elevation model for the US

item_dem = gis.content.search('elevation_270m')[0]
lyr_dem = item_dem.layers[0]

In [ ]:
# Human Modification Index imagery layer
# This dataset is based on research on the degree of human modification to
# the landscape, on a scale of 0 - 1, where 0.0 indicates unmodified natural
# landscape and 1.0 indicates the landscape is completely modified by human activity.  
item_hmi = gis.content.search('human_modification_index')[0]
lyr_hmi = item_hmi.layers[0]

In [ ]:
# geocode the area of interest
from arcgis.geocoding import geocode

sd_county = geocode('San Diego County', 
                    out_sr=lyr_dem.properties.spatialReference)[0]

In [ ]:
# set the area of interest as the layer's extent
lyr_dem.extent = sd_county['extent']

In [ ]:
lyr_hmi.extent = sd_county['extent']

In [ ]:
# Create a colormap for rendering the analysis results.
red_green =  [[1,  38, 115,0],[2,  86, 148,0],[3,  0x8B, 0xB5,0],[4,  0xC5, 0xDB,0],
         [5, 255, 255, 0],[6,  0xFF, 0xC3,0],[7, 0xFA, 0x8E, 0],[8,  0xF2, 0x55,0],
         [9, 0xE6, 0, 0]]

In [ ]:
from arcgis.raster.functions import *

In [ ]:
output_values = [1,2,3,4,5,6,7,8,9]

In [ ]:
elevation = remap(lyr_dem,  # Elevation 
                    [-90,250, 250,500, 500,750, 750,1000, 1000,1500, 1500,2000, 2000,2500, 2500,3000, 3000,5000],
                    output_values) 

In [ ]:
natural_areas = remap(lyr_hmi, # Human Modified Index 
                     [0.0,0.1, 0.1,0.2, 0.2,0.3, 0.3,0.4, 0.4,0.5,0.5,0.6, 0.6,0.7, 0.7,0.8, 0.8,1.1],
                     output_values)

In [ ]:
terrain = remap(slope(lyr_dem, slope_type='DEGREE', z_factor=1), # Slope
                     [0,1, 1,2, 2,3, 3,5, 5,7, 7,9, 9,12, 12,15, 15,100],
                     output_values) 

# Analysis

In [ ]:
# Map algebra for the web GIS!
result = 0.2*elevation + 0.3*terrain + 0.5*natural_areas

In [ ]:
run_raster = colormap(clip(result, sd_geom), colormap=red_green)
run_raster

In [ ]:
runmap = gis.map('San Diego, CA', zoomlevel=12)
runmap.add_layer(run_raster, {'opacity': 0.75})
runmap

### Persist Results

In [ ]:
# Persist results as an imagery layer using distributed raster analysis (Image Server)
# run_raster.save('SanDiego_PlacesToRun')